# Sistema de Respuestas Automatizadas para MercadoLibre
## Análisis Comparativo de Modelos LLM con Sistema de Caché

Este notebook implementa y evalúa un sistema de respuestas automatizadas para preguntas de clientes en MercadoLibre, utilizando diferentes modelos de OpenAI y estrategias de prompting, con un sistema de caché basado en embeddings para optimizar costos y tiempo de respuesta.

In [9]:
# --- Imports ---
import pandas as pd
import os
import time
from IPython.display import display
from tqdm.notebook import tqdm
import chromadb
from langchain_openai import ChatOpenAI
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_core.messages import SystemMessage, HumanMessage
from ragas import evaluate
from ragas.metrics import faithfulness
from datasets import Dataset

# --- Configuración de la API Key ---
api_key = os.getenv('OPENAI_API_KEY')
if not api_key:
    print("ADVERTENCIA: No se encontró la API Key de OpenAI. El LLM no funcionará.")

# --- Inicialización de Embeddings ---
embedding_model = HuggingFaceEmbeddings(
    model_name='sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2',
    model_kwargs={'device': 'cpu'}
)
print("Modelo de embeddings inicializado.")

# --- MODIFICACIÓN: Función para inicializar LLMs dinámicamente ---
def inicializar_llm(model_name: str):
    """Inicializa un modelo de ChatOpenAI con el nombre especificado."""
    if not api_key:
        return None
    try:
        llm = ChatOpenAI(
            model=model_name,
            openai_api_key=api_key,
            temperature=0.1
        )
        print(f"Modelo LLM '{model_name}' inicializado.")
        return llm
    except Exception as e:
        print(f"Error al inicializar el modelo '{model_name}': {e}")
        return None

# --- NUEVA FUNCIÓN: Guardar resultados en CSV ---
def guardar_resultados_csv(results_df: pd.DataFrame, modelo: str, prompt_name: str, output_dir: str = "../results"):
    """Guarda los resultados en CSV con el formato especificado: site|pregunta|publicación|respuesta generada"""
    
    # Crear directorio si no existe
    os.makedirs(output_dir, exist_ok=True)
    
    # Preparar DataFrame con la estructura requerida
    csv_df = pd.DataFrame({
        'site': 'MercadoLibre',  # Valor fijo para MercadoLibre
        'pregunta': results_df['pregunta'],
        'publicacion': results_df['publicacion'],
        'respuesta_generada': results_df['respuesta_generada']
    })
    
    # Nombre del archivo
    filename = f"{modelo}_{prompt_name}_resultados.csv"
    filepath = os.path.join(output_dir, filename)
    
    # Guardar con separador |
    csv_df.to_csv(filepath, sep='|', index=False, encoding='utf-8-sig')
    print(f"Resultados guardados en: {filepath}")
    
    return filepath

print("\n✅ Configuración inicial completada y lista para inicializar modelos dinámicamente.")
print("✅ Función de guardado CSV agregada.")

Modelo de embeddings inicializado.

✅ Configuración inicial completada y lista para inicializar modelos dinámicamente.
✅ Función de guardado CSV agregada.


## Carga y Exploración de Datos

Se carga el dataset de preguntas reales de MercadoLibre para evaluar el sistema con casos de uso reales.

In [10]:
# --- Carga y exploración inicial de los datos ---

# Ruta al archivo Excel
file_path = "..\data\preguntas_mercadolibre.xlsx" # Ajusta la ruta si es necesario

try:
    df = pd.read_excel(file_path)
    print(f"Dataset cargado exitosamente desde '{file_path}'.")
except FileNotFoundError:
    print(f"Error: El archivo no se encontró")

# --- Exploración básica de los datos ---
print(f"\nEl dataset tiene {df.shape[0]} filas y {df.shape[1]} columnas.")
print("Primeras filas del dataset:")
display(df.head())

# --- MODIFICACIÓN: Usaremos el DataFrame completo para el experimento ---
df_procesar = df.copy()
print(f"\nSe procesará el dataset completo: {len(df_procesar)} filas.")

Dataset cargado exitosamente desde '..\data\preguntas_mercadolibre.xlsx'.

El dataset tiene 8 filas y 3 columnas.
Primeras filas del dataset:


<>:4: SyntaxWarning: invalid escape sequence '\d'
<>:4: SyntaxWarning: invalid escape sequence '\d'
C:\Users\andre\AppData\Local\Temp\ipykernel_22432\386372061.py:4: SyntaxWarning: invalid escape sequence '\d'
  file_path = "..\data\preguntas_mercadolibre.xlsx" # Ajusta la ruta si es necesario


,SITE,PREGUNTA,PUBLICACION
0,MLC,"Hola! La mascarilla es de 500 ml, es la origin...",title: Mascarilla Para El Cabello Karseell Col...
1,MLM,"Hola, es nuevo, facturan, tiene garantía?",title: Cámara Termográfica Flir Tg297 Point 19...
2,MLC,"Hola, vienen con caja? Son de buena calidad? T...",title: Zapatos Amiu Nuevos De Verano 2024 Para...
3,MLB,Vocês emitem nota fiscal da compra ?,title: Rodizio Borracha Roda Maciça 250-4 Rol....
4,MLM,Hice compra y quisiera factura,title: Manguera De Jardín Expandible De Hasta ...



Se procesará el dataset completo: 8 filas.


## Definición de Estrategias de Prompting

Se implementan tres enfoques diferentes para responder preguntas de clientes:

1. **Preciso**: Responde únicamente con información disponible en la publicación
2. **Colaborador**: Prioriza la información de la publicación pero puede ofrecer conocimiento general 
3. **Proactivo**: Enfoque de ventas que destaca características positivas del producto

In [11]:
# --- Diccionario de Plantillas de Prompts ---

PROMPTS = {
    "preciso": """
**ROL Y OBJETIVO:**
Actuarás como un asistente virtual experto para Mercado Libre. Tu única tarea es responder la PREGUNTA DEL CLIENTE usando exclusivamente la INFORMACIÓN DE LA PUBLICACIÓN.

**REGLAS:**
1.  **Fuente Única:** Basa tu respuesta 100% en la `INFORMACIÓN DE LA PUBLICACIÓN`. No inventes ni supongas nada.
2.  **Información Ausente:** Si la respuesta no está en el texto, tu única respuesta debe ser: "¡Hola! Gracias por tu pregunta. 🤔 Esa información no está especificada en la publicación, ¡lo siento!".
3.  **Tono:** Amable, conciso y directo. Usa emojis apropiados (👋, 🤔, 😊).
4.  **Idioma:** Responde siempre en el mismo idioma de la pregunta.
---
**INFORMACIÓN DE LA PUBLICACIÓN:**
{contexto}
""",

    "colaborador": """
**ROL Y OBJETIVO:**
Actuarás como un asistente virtual colaborativo para Mercado Libre. Tu objetivo es ser lo más útil posible, priorizando la INFORMACIÓN DE LA PUBLICACIÓN.

**REGLAS:**
1.  **Prioridad del Contexto:** Busca siempre la respuesta en la `INFORMACIÓN DE LA PUBLICACIÓN` primero.
2.  **Información Ausente:** Si la respuesta no está, admítelo ("Esa información no está detallada en la publicación.") y luego ofrece conocimiento general, indicando que es una suposición ("Sin embargo, generalmente en productos de este tipo...").
3.  **Tono:** Servicial, positivo y amable. Usa emojis (✨, 😉, 👍).
4.  **Idioma:** Responde siempre en el mismo idioma de la pregunta.
---
**INFORMACIÓN DE LA PUBLICACIÓN:**
{contexto}
""",

    "proactivo": """
**ROL Y OBJETIVO:**
Actuarás como un asistente de ventas proactivo para Mercado Libre. Tu misión es mantener el interés del cliente y destacar ventajas del producto.

**REGLAS:**
1.  **Base en el Contexto:** Tus afirmaciones deben originarse en la `INFORMACIÓN DE LA PUBLICACIÓN`.
2.  **Información Ausente:** Si la respuesta no está, no adivines. En su lugar, redirige a una característica positiva que sí esté en el contexto ("Ese detalle no está especificado, pero ¡mira esto!, cuenta con [característica positiva]. 🤩") o sugiere preguntar directamente al vendedor.
3.  **Tono:** Entusiasta, positivo y cercano. Usa emojis que generen emoción (🎉, 🤩, 🚀).
4.  **Idioma:** Responde siempre en el mismo idioma de la pregunta.
---
**INFORMACIÓN DE LA PUBLICACIÓN:**
{contexto}
"""
}

print("Diccionario de prompts ('preciso', 'colaborador', 'proactivo') creado.")

Diccionario de prompts ('preciso', 'colaborador', 'proactivo') creado.


## Sistema de Caché Basado en Embeddings

Implementación de un sistema de caché inteligente que utiliza ChromaDB y embeddings semánticos para:
- Evitar llamadas redundantes a los LLMs
- Reducir costos operativos
- Mejorar tiempo de respuesta
- Reutilizar respuestas para preguntas similares en la misma publicación

In [12]:
# --- Configuración del Cliente y Colección de ChromaDB ---
client = chromadb.Client()
collection_name = "qa_cache_mercadolibre_v3" # Usamos v3 para evitar conflictos
collection = client.get_or_create_collection(
    name=collection_name,
    metadata={"hnsw:space": "cosine"}
)

# --- Contadores y Mapeos para IDs ---
publication_to_id_map = {}
next_publication_id = 0
next_cache_entry_id = 0

def get_publication_id(publication_text: str) -> str:
    global next_publication_id
    if publication_text not in publication_to_id_map:
        publication_to_id_map[publication_text] = str(next_publication_id)
        next_publication_id += 1
    return publication_to_id_map[publication_text]

def search_cache(pregunta: str, publication_id: str, similarity_threshold: float = 0.95) -> str | None:
    if collection.count() == 0: return None
    pregunta_embedding = embedding_model.embed_query(pregunta)
    results = collection.query(
        query_embeddings=[pregunta_embedding],
        n_results=1,
        where={"publication_id": publication_id}
    )
    if results['ids'][0]:
        similarity = 1 - results['distances'][0][0]
        if similarity > similarity_threshold:
            return results['metadatas'][0][0]['respuesta']
    return None

def add_to_cache(pregunta: str, respuesta: str, publication_id: str):
    global next_cache_entry_id
    pregunta_embedding = embedding_model.embed_query(pregunta)
    entry_id = str(next_cache_entry_id)
    collection.upsert(
        ids=[entry_id],
        embeddings=[pregunta_embedding],
        metadatas=[{
            "publication_id": publication_id,
            "pregunta": pregunta,
            "respuesta": respuesta
        }]
    )
    next_cache_entry_id += 1

# La limpieza de la colección se hará al inicio de cada experimento en el orquestador.
print(f"Sistema de caché configurado. Colección '{collection_name}' lista.")

Sistema de caché configurado. Colección 'qa_cache_mercadolibre_v3' lista.


## Generación de Respuestas con LLM

Función principal que utiliza los modelos de OpenAI para generar respuestas contextualmente relevantes.

In [13]:
# --- MODIFICACIÓN: La función ahora recibe el objeto LLM ---
def generar_respuesta_llm(llm: ChatOpenAI, pregunta: str, publicacion: str, prompt_template: str):
    """Genera una respuesta utilizando el LLM especificado."""
    if not llm:
        return "Error: El modelo LLM no fue proporcionado o no está inicializado."

    system_prompt = prompt_template.format(contexto=publicacion)
    messages = [SystemMessage(content=system_prompt), HumanMessage(content=pregunta)]
    
    try:
        response = llm.invoke(messages)
        return response.content.strip()
    except Exception as e:
        return f"Error al llamar al modelo: {e}"

print("Función 'generar_respuesta_llm' actualizada.")

Función 'generar_respuesta_llm' actualizada.


## Evaluación con RAGAS

Implementación de evaluación automática usando RAGAS (Retrieval Augmented Generation Assessment) para medir la calidad de las respuestas generadas.

In [14]:
# --- MODIFICACIÓN: La función ahora recibe el objeto LLM ---
def evaluar_con_ragas(llm: ChatOpenAI, results_df: pd.DataFrame):
    """Evalúa un DataFrame usando RAGAS con el LLM especificado."""
    if not llm:
        print("No se puede evaluar con RAGAS porque el LLM no fue proporcionado.")
        return pd.DataFrame()

    dataset = Dataset.from_dict({
        "question": results_df["pregunta"].tolist(),
        "answer": results_df["respuesta_generada"].tolist(),
        "contexts": [[c] for c in results_df["publicacion"].tolist()]
    })
    
    result = evaluate(dataset=dataset, metrics=[faithfulness], llm=llm, embeddings=embedding_model)
    return result.to_pandas()

print("Función 'evaluar_con_ragas' actualizada.")

Función 'evaluar_con_ragas' actualizada.


## Orquestador de Experimentos

Función principal que coordina todo el flujo: generación de respuestas, uso de caché, y evaluación de resultados.

In [15]:
# --- MODIFICACIÓN: La función ahora recibe el objeto LLM y guarda resultados en CSV ---
def ejecutar_experimento(llm: ChatOpenAI, df: pd.DataFrame, prompt_template: str, modelo_name: str = None, prompt_name: str = None):
    """Orquesta el flujo completo de generación de respuestas y guarda resultados en CSV."""
    results_list = []
    
    # Reiniciar el caché para un experimento limpio
    if collection.count() > 0:
        collection.delete(ids=collection.get()['ids'])
    global next_publication_id, publication_to_id_map, next_cache_entry_id
    next_publication_id, next_cache_entry_id = 0, 0
    publication_to_id_map = {}
    
    print(f"Iniciando experimento para el modelo '{llm.model_name}' con {len(df)} filas...")
    for _, row in tqdm(df.iterrows(), total=df.shape[0], desc=f"Generando con {llm.model_name}"):
        pregunta, publicacion = row['PREGUNTA'], row['PUBLICACION']
        publication_id = get_publication_id(publicacion)
        
        cached_response = search_cache(pregunta, publication_id)
        
        if cached_response:
            source, respuesta_final = "Cache", cached_response
        else:
            source = "LLM"
            respuesta_final = generar_respuesta_llm(llm, pregunta, publicacion, prompt_template)
            add_to_cache(pregunta, respuesta_final, publication_id)
            time.sleep(1)

        results_list.append({
            'pregunta': pregunta, 'publicacion': publicacion,
            'respuesta_generada': respuesta_final, 'fuente_respuesta': source,
        })

    results_df = pd.DataFrame(results_list)
    
    # Guardar resultados en CSV si se proporcionan los nombres
    if modelo_name and prompt_name:
        guardar_resultados_csv(results_df, modelo_name, prompt_name)
    
    return results_df

print("Función 'ejecutar_experimento' actualizada para guardar resultados en CSV.")

Función 'ejecutar_experimento' actualizada para guardar resultados en CSV.


## Benchmark Comparativo de Modelos

Evaluación sistemática de diferentes combinaciones de modelos GPT y estrategias de prompting para identificar la configuración óptima.

In [16]:
# --- 1. DEFINICIÓN DEL BENCHMARK ---
modelos_a_probar = ["gpt-4.1-nano", 'gpt-4.1-mini', 'gpt-4.1']
prompts_a_probar = ["preciso", "colaborador", "proactivo"]

summary_results = []

# --- 2. BUCLE DEL BENCHMARK ---
for model_name in modelos_a_probar:
    llm = inicializar_llm(model_name)
    if not llm: continue

    for prompt_name in prompts_a_probar:
        template = PROMPTS[prompt_name]
        
        # Usar la función existente ejecutar_experimento con guardado CSV
        results_df = ejecutar_experimento(llm, df_procesar, template, model_name, prompt_name)
        
        llm_results = results_df[results_df['fuente_respuesta'] == 'LLM'].copy()
        
        if not llm_results.empty:
            # Evaluar con RAGAS
            ragas_results_df = evaluar_con_ragas(llm, llm_results)
            avg_faithfulness = ragas_results_df['faithfulness'].mean() if 'faithfulness' in ragas_results_df.columns else 0.0
        else:
            avg_faithfulness = 0.0
        
        summary_results.append({
            'modelo': model_name,
            'prompt': prompt_name,
            'faithfulness_score': round(avg_faithfulness, 4)
        })

# --- 3. TABLA COMPARATIVA DEL BENCHMARK ---
if summary_results:
    summary_df = pd.DataFrame(summary_results).sort_values(by='faithfulness_score', ascending=False)
    summary_df.to_csv("../results/resumen_benchmark.csv", index=False, encoding='utf-8-sig')
    display(summary_df)


Modelo LLM 'gpt-4.1-nano' inicializado.
Iniciando experimento para el modelo 'gpt-4.1-nano' con 8 filas...


Generando con gpt-4.1-nano:   0%|          | 0/8 [00:00<?, ?it/s]

Resultados guardados en: ../results\gpt-4.1-nano_preciso_resultados.csv


Evaluating:   0%|          | 0/8 [00:00<?, ?it/s]

Iniciando experimento para el modelo 'gpt-4.1-nano' con 8 filas...


...

Iniciando experimento para el modelo 'gpt-4.1' con 8 filas...


Generando con gpt-4.1:   0%|          | 0/8 [00:00<?, ?it/s]

Resultados guardados en: ../results\gpt-4.1_proactivo_resultados.csv


Evaluating:   0%|          | 0/8 [00:00<?, ?it/s]

,modelo,prompt,faithfulness_score
1,gpt-4.1-nano,colaborador,0.7354
5,gpt-4.1-mini,proactivo,0.7335
2,gpt-4.1-nano,proactivo,0.6910
4,gpt-4.1-mini,colaborador,0.6867
8,gpt-4.1,proactivo,0.6486
7,gpt-4.1,colaborador,0.6293
3,gpt-4.1-mini,preciso,0.6129
6,gpt-4.1,preciso,0.5542
0,gpt-4.1-nano,preciso,0.4250


## Pruebas del Sistema de Caché

Validación del funcionamiento del sistema de caché con datos reales del dataset.

In [17]:
# --- MODIFICACIÓN: La función de prueba ahora recibe el ID de la publicación ---

def probar_cache_con_id(pregunta: str, publicacion: str, publication_id: str, llm: ChatOpenAI, prompt_template: str):
    """
    Prueba el sistema de caché usando directamente el ID de la publicación.

    Args:
        pregunta (str): La pregunta del cliente.
        publicacion (str): El texto de la publicación (necesario para el LLM si hay un cache miss).
        publication_id (str): El ID único de la publicación.
        llm (ChatOpenAI): El objeto LLM para generar respuestas.
        prompt_template (str): La plantilla de prompt.

    Returns:
        tuple[str, str]: Una tupla con la respuesta y la fuente ('Cache' o 'LLM').
    """
    # 1. Buscar en el caché usando el ID
    cached_response = search_cache(pregunta, publication_id)
    
    if cached_response:
        # Cache Hit
        return cached_response, "Cache"
    else:
        # Cache Miss
        # 2. Generar respuesta si no está en el caché
        nueva_respuesta = generar_respuesta_llm(llm, pregunta, publicacion, prompt_template)
        # 3. Guardar en el caché usando el ID
        add_to_cache(pregunta, nueva_respuesta, publication_id)
        return nueva_respuesta, "LLM"

print("Función 'probar_cache_con_id' definida, que acepta el ID directamente.")

Función 'probar_cache_con_id' definida, que acepta el ID directamente.


In [ ]:
print("Usando datos reales del DataFrame 'df_procesar' para la prueba.")


fila_de_prueba = df_procesar.iloc[0]
pregunta_real = fila_de_prueba['PREGUNTA']
publicacion_real = fila_de_prueba['PUBLICACION']

print("\nDatos de prueba extraídos de la primera fila del dataset:")
print(f"  - Pregunta: '{pregunta_real}'")

id_publicacion_real = get_publication_id(publicacion_real)
print(f"\nID obtenido para la publicación: '{id_publicacion_real}'")

prompt = PROMPTS['preciso'] 

print("\n--- PRIMERA LLAMADA ---")
respuesta1, fuente1 = probar_cache_con_id(pregunta_real, publicacion_real, id_publicacion_real, llm, prompt)
print(f"Fuente: {fuente1}")
print(f"Respuesta: {respuesta1}")

print("\n--- SEGUNDA LLAMADA ---")
respuesta2, fuente2 = probar_cache_con_id("¿Qué otros productos tienes?", publicacion_real, id_publicacion_real, llm, prompt)
print(f"Fuente: {fuente2}")
print(f"Respuesta: {respuesta2}")

print("\n--- TERCERA LLAMADA ---")
respuesta3, fuente3 = probar_cache_con_id("¿Qué otros productos tienes?", publicacion_real, id_publicacion_real, llm, prompt)
print(f"Fuente: {fuente3}")
print(f"Respuesta: {respuesta3}")

Usando datos reales del DataFrame 'df_procesar' para la prueba.

Datos de prueba extraídos de la primera fila del dataset:
  - Pregunta: 'Hola! La mascarilla es de 500 ml, es la original? Y tiene algún contacto donde le pueda realizar algunas preguntas'

ID obtenido para la publicación: '0'

--- PRIMERA LLAMADA ---
Fuente: Cache
Respuesta: ¡Hola! 🤩 Sí, la mascarilla para el cabello Karseell Collagen que ves en la publicación es de 500 ml, tal como lo indica el título. Sobre si es la original, ese detalle específico no está mencionado en la información de la publicación, pero ¡mira esto! Es un producto nuevo, viene en presentación individual y cuenta con garantía del vendedor por 60 días, lo que te da mayor confianza en tu compra. 🎉

Respecto a un contacto directo, Mercado Libre recomienda que todas las consultas se realicen a través de la sección de preguntas de la publicación, ¡así el vendedor puede responderte de forma segura y rápida! Si tienes más dudas, ¡aquí estoy para ayudarte! 

El sistema de caché está pensado para evitar redundancia sobre la misma pregunta. La intuición me dice que es muy probable que una gran proporción de las preguntas que se reciben sean repetitivas y fundamentalmente iguales, por lo que este sistema tiene sentido.

# Conclusiones y Hallazgos

## Sistema de Caché Basado en Embeddings

Dado el gran volumen de preguntas que recibe MercadoLibre semanalmente, tiene sentido económico implementar un sistema de caché inteligente. El sistema propuesto utiliza embeddings semánticos para identificar preguntas similares sobre la misma publicación y reutilizar respuestas previamente generadas por LLMs, optimizando tanto costos como tiempo de respuesta.

## Selección de Modelo Óptimo (Corregido)

Basado en los resultados del benchmark de faithfulness score:

**Recomendación**: **GPT-4.1-nano con prompt "colaborador"** (0.7354)
- Mayor fidelidad a la información de la publicación.
- Reduce el riesgo de alucinaciones.
- Modelo más económico ("nano") con el mejor rendimiento.

**Alternativa de Alto Rendimiento**: **GPT-4.1-mini con prompt "proactivo"** (0.7335)
- Performance casi idéntica al modelo recomendado.
- Excelente balance entre calidad y costo.
- Ideal si se busca una respuesta más proactiva sin sacrificar fidelidad.

## Análisis de Costos

**Precios de inferencia por 1M tokens:**
- GPT-4.1: Input $3.00, Output $12.00
- GPT-4.1-mini: Input $0.80, Output $3.20  
- GPT-4.1-nano: Input $0.20, Output $0.80

El sistema de caché puede reducir de manera significativa las llamadas a LLM para preguntas frecuentes, generando ahorros sustanciales. Sin embargo, debe considerarse el costo de hospedar la base de datos vectorial (ChromaDB/Pinecone) versus los ahorros en tokens de LLM.

## Evaluación con RAGAS

Se implementó RAGAS para establecer un estándar de evaluación objetiva. Se utilizó la métrica "faithfulness" por simplicidad, pero existen múltiples métricas adicionales (answer_relevancy, context_precision, etc.) que deberían alinearse con objetivos específicos del negocio.

Es importante destacar que esta es una forma automatizada de evaluar LLMs con otros sistemas de LLMs, que también están sujetos a variaciones y a incertidumbre. La definición de una métrica adecuada suele ir acompañada de una alineación previa con negocio. 

Por otro lado, existen muchas otras formas de evaluar el rendimiento de un LLM en este tipo de tareas. Elegí la anterior por simplicidad.

## Archivos de Resultados Generados

El sistema ahora genera automáticamente archivos CSV con la estructura solicitada:
- **Formato**: `site|pregunta|publicacion|respuesta_generada`
- **Ubicación**: `../results/`
- **Nomenclatura**: `{modelo}_{prompt}_resultados.csv`

Cada combinación de modelo y prompt genera un archivo CSV independiente para facilitar el análisis posterior.

## Próximos Pasos

### Gestión de Problemas Identificados:

1. **Alucinaciones**: 
   - Validación automática de respuestas contra contexto de publicación.
   - Evaluación continua antes de la respuesta. 
   - Tamaño del modelo.

2. **Información Contradictoria**:
   - Sistema de versionado de publicaciones en el caché
   - Invalidación automática cuando se actualiza una publicación para el sistema de Caché. 

3. **Ausencia de Datos**:
   - Respuestas predefinidas que admiten limitaciones: "Esa información no está especificada"
   - Redirección a contacto directo con vendedor o con un agente especializado.
   - Incentivo a completar la descripción de forma oportuna. 

4. **Optimización del Sistema**:
   - Implementar métricas de negocio específicas (conversión, satisfacción)
   - A/B testing entre diferentes estrategias de prompting y percepción de un usuario final.
   - Monitoreo continuo de calidad y costos.

La solución propuesta representa un enfoque balanceado entre calidad de respuestas, eficiencia operativa y control. Sin embargo, tiene muchas oportunidades de mejora.